### 1. Data Processing
#### 1.1 Data Load

In [2]:
train_df = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/train.csv')
building_metadata_df = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/building_metadata.csv')
weather_train_df = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/weather_train.csv')

#### 1.2 Data Processing

In [4]:
from pyspark.sql.functions import when, avg, col, round
from pyspark.sql.window import Window

In [5]:
### weather train dataset processing ###

w = Window().partitionBy('site_id')

#Replace negative values of 'qty' with Null, as we don't want to consider them while averaging.
#weather_train_df = weather_train_df.withColumn('qty',when(col('qty')<0,None).otherwise(col('qty')))
weather_train_df = weather_train_df.withColumn('cloud_coverage',when(col('cloud_coverage').isNull(),avg(col('cloud_coverage')).over(w)).otherwise(col('cloud_coverage')))

weather_train_df = weather_train_df.withColumn("cloud_coverage", round(weather_train_df["cloud_coverage"]).cast('integer'))

weather_train_df = weather_train_df.withColumn('sea_level_pressure',when(col('sea_level_pressure').isNull(),avg(col('sea_level_pressure')).over(w)).otherwise(col('sea_level_pressure')))

weather_train_df = weather_train_df.withColumn('precip_depth_1_hr',when(col('precip_depth_1_hr').isNull(),avg(col('precip_depth_1_hr')).over(w)).otherwise(col('precip_depth_1_hr')))
weather_train_df = weather_train_df.withColumn("precip_depth_1_hr", round(weather_train_df["precip_depth_1_hr"]).cast('integer'))

weather_train_df=weather_train_df.na.drop()

In [6]:
### building meta data processing ###

w = Window().partitionBy('primary_use')
building_metadata_df = building_metadata_df.withColumn('year_built',when(col('year_built').isNull(),avg(col('year_built')).over(w)).otherwise(col('year_built')))
building_metadata_df = building_metadata_df.withColumn("year_built", round(building_metadata_df["year_built"]).cast('integer'))

building_metadata_df = building_metadata_df.withColumn('floor_count',when(col('floor_count').isNull(),avg(col('floor_count')).over(w)).otherwise(col('floor_count')))
building_metadata_df = building_metadata_df.withColumn("floor_count", round(building_metadata_df["floor_count"]).cast('integer'))

building_metadata_df=building_metadata_df.na.drop()

building_metadata_df = building_metadata_df.withColumn("primary_use", when(col("primary_use")=="Education", 1)
                                   .when(col("primary_use")=="Office", 2)
                                   .when(col("primary_use")=="Entertainment/public assembly", 3)
                                   .when(col("primary_use")=="Public services", 4)
                                   .when(col("primary_use")=="Lodging/residential", 5)
                                   .when(col("primary_use")=="Other", 6)
                                   .when(col("primary_use")=="Healthcare", 7)
                                   .when(col("primary_use")=="Parking", 8)
                                   .when(col("primary_use")=="Warehouse/storage", 9)
                                   .when(col("primary_use")=="Manufacturing/industrial", 10)
                                   .when(col("primary_use")=="Retail", 11)
                                   .when(col("primary_use")=="Services", 12)
                                   .when(col("primary_use")=="Technology/science", 13)
                                   .when(col("primary_use")=="Food sales and service", 14)
                                   .when(col("primary_use")=="Utility", 15)
                                   .when(col("primary_use")=="Religious worship", 16))

In [7]:
### train dataset processing ###
train_df = train_df.withColumn("meter_reading", when(col("meter")==0, col("meter_reading")*0.293))

In [8]:
# test_df = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/test.csv')
# building_metadata_df = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/building_metadata.csv')
# weather_test_df = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/weather_test.csv')

In [9]:
meta_train_df =  building_metadata_df.join(train_df, (building_metadata_df['building_id'] == train_df['building_id']))
cond = [weather_train_df.site_id == meta_train_df.site_id, weather_train_df.timestamp == meta_train_df.timestamp]
trainDF =  weather_train_df.join(meta_train_df, cond)

In [10]:
# meta_test_df =  building_metadata_df.join(test_df, (building_metadata_df['building_id'] == test_df['building_id']))
# cond = [weather_test_df.site_id == meta_test_df.site_id, weather_test_df.timestamp == meta_test_df.timestamp]
# testDF =  weather_test_df.join(meta_test_df, cond)

In [11]:
datasetDF = trainDF.drop("timestamp", "site_id", "building_id")
datasetDF = datasetDF.na.fill(0)

In [12]:
# datasetTestDF = testDF.drop("timestamp", "site_id", "building_id")
# datasetTestDF = datasetTestDF.na.fill(0)

In [13]:
split15DF, split85DF = datasetDF.randomSplit([15., 85.], seed=190)

# Let's cache these datasets for performance
testSetDF = split15DF.cache()
trainingSetDF = split85DF.cache()

### 2. ML and Evaluation Set Up

In [15]:
from pyspark.ml import Pipeline
from pyspark.sql.functions import log
from pyspark.ml.evaluation import Evaluator, RegressionEvaluator
from math import sqrt
from statistics import mean
# cross validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# ***** vectorizer MODEL ****
from pyspark.ml.feature import VectorAssembler

vectorizer = VectorAssembler()
vectorizer.setInputCols(["air_temperature", "cloud_coverage", "dew_temperature", "precip_depth_1_hr", "sea_level_pressure", 
                         "wind_direction", "wind_speed", "square_feet", "year_built", "floor_count", "meter", "primary_use"])
vectorizer.setOutputCol("features")

class RMSLEEvaluator(Evaluator):

    def __init__(self, predictionCol="prediction", labelCol="label"):
        self.predictionCol = predictionCol
        self.labelCol = labelCol

    def _evaluate(self, dataset):
        """
        Returns a random number. 
        Implement here the true metric
        """
        new_dataset = dataset.withColumn('result_'+self.predictionCol, log(col(self.predictionCol)+1))
        new_dataset = new_dataset.withColumn('result_'+self.labelCol, log(col(self.labelCol)+1))
        new_dataset = new_dataset.withColumn('result', (col('result_'+self.predictionCol) - col('result_'+self.labelCol))**2)
        
        result = new_dataset.agg(avg(col("result")))
        result = result.collect()[0]["avg(result)"]
        return sqrt(result)
      
    def isLargerBetter(self):
        return True

#### 2.1 Linear Regression
##### 2.1.1 Linear Regression Pipeline

In [17]:
# ***** LINEAR REGRESSION MODEL ****
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel
# Let's initialize our linear regression learner
lr = LinearRegression()

# Now we set the parameters for the method
lr.setPredictionCol("predicted_meter_reading")\
  .setLabelCol("meter_reading")\
  .setMaxIter(100)\
  .setRegParam(0.15)

# We will use the new spark.ml pipeline API. If you have worked with scikit-learn this will be very familiar.
lrPipeline = Pipeline()

lrPipeline.setStages([vectorizer, lr])

Out[81]: Pipeline_2005b69b9767

#### 2.1.2 Linear Regression Simple Fitting

In [19]:
# Let's first train on the entire dataset to see what we get
lrModel = lrPipeline.fit(trainingSetDF)

#### 2.1.3 Linear Regression Simple Evaluation

In [21]:
# The intercept is as follows:
intercept = lrModel.stages[1].intercept

# The coefficents (i.e., weights) are as follows:
weights = lrModel.stages[1].coefficients

# Create a list of the column names (without PE)
featuresNoLabel = [col for col in datasetDF.columns if col != "meter_reading" and col != "primary_use"]

# Merge the weights and labels
coefficents = zip(weights, featuresNoLabel)

equation = "y = {intercept}".format(intercept=intercept)
variables = []
for x in coefficents:
    weight = abs(x[0])
    name = x[1]
    symbol = "+" if (x[0] > 0) else "-"
    equation += (" {} ({} * {})".format(symbol, weight, name))

# Finally here is our equation
print("Linear Regression Equation: " + equation)

resultsDF = lrModel.transform(testSetDF)
# Create an RMSE evaluator using the label and predicted columns
regEval = RMSLEEvaluator(predictionCol="predicted_meter_reading", labelCol="meter_reading")

# Run the evaluator on the DataFrame
rmsle = regEval.evaluate(resultsDF)

print("Simple Root Mean Squared Logarithmtic Error: %.2f" % rmsle)

Linear Regression Equation: y = -161.49270517166903 - (0.024079192795500377 * air_temperature) + (0.13825604358664045 * cloud_coverage) + (0.08693710452724579 * dew_temperature) - (0.002700260347343527 * precip_depth_1_hr) + (0.04279355860320019 * sea_level_pressure) + (0.0014832816396364843 * wind_direction) + (0.0865212554352526 * wind_speed) + (7.245281257821358e-05 * square_feet) + (0.06123188029759769 * year_built) + (0.8976686419093083 * floor_count) - (7.5292296456378 * meter)
Simple Root Mean Squared Logarithmtic Error: 1.45

#### 2.1.4 Linear Regression Cross Validation Fitting

In [23]:
# We can reuse the RegressionEvaluator, regEval, to judge the model based on the best Root Mean Squared Error
# Let's create our CrossValidator with 3 fold cross validation
crossval = CrossValidator(estimator=lrPipeline, evaluator=regEval, numFolds=3)

# Let's tune over our regularization parameter from 0.05 to 0.50
regParam = [x / 200.0 for x in range(1, 11)]

# We'll create a paramter grid using the ParamGridBuilder, and add the grid to the CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, regParam)
             .build())
crossval.setEstimatorParamMaps(paramGrid)

# Now let's find and return the best model
cvModel = crossval.fit(trainingSetDF).bestModel

#### 2.1.5 Linear Regression Cross Validation Evaluation

In [25]:
# evaluation
resultsDF = lrModel.transform(testSetDF)
# Create an RMSE evaluator using the label and predicted columns
regEval = RMSLEEvaluator(predictionCol="predicted_meter_reading", labelCol="meter_reading")

# Run the evaluator on the DataFrame
rmsle = regEval.evaluate(resultsDF)

print("Cross Validated Root Mean Squared Logarithmtic Error: %.2f" % rmsle)

### 2.2 Decision Tree
#### 2.2.1 Decision Tree Pipeline

In [27]:
from pyspark.ml.regression import DecisionTreeRegressor

# Create a DecisionTreeRegressor
dt = DecisionTreeRegressor()

dt.setPredictionCol("predicted_meter_reading")\
  .setLabelCol("meter_reading")\
  .setFeaturesCol("features")\
  .setMaxBins(100)

# Create a Pipeline
dtPipeline = Pipeline()

# Set the stages of the Pipeline
dtPipeline.setStages([vectorizer, dt])

#### 2.2.2 Decision Tree Simple Fitting

In [29]:
# Let's first train on the entire dataset to see what we get
dtModel = dtPipeline.fit(trainingSetDF)

#### 2.2.3 Decision Tree Simple Evaluation

In [31]:
resultsDF = dtModel.transform(testSetDF)
# Create an RMSE evaluator using the label and predicted columns
regEval = RMSLEEvaluator(predictionCol="predicted_meter_reading", labelCol="meter_reading")

# Run the evaluator on the DataFrame
rmsle = regEval.evaluate(resultsDF)

print("Simple Root Mean Squared Logarithmtic Error: %.2f" % rmsle)

# dtSubmitted = dtModel.transform(datasetTestDF)
# dtSubmitted = dtSubmitted.select('row_id', 'predicted_meter_reading')
# dtSubmitted = dtSubmitted.withColumnRenamed('predicted_meter_reading', "meter_reading")

In [32]:
# file='/FileStore/tables/my.csv'
# dtSubmitted = dtSubmitted.repartition(1)
   
# dtSubmitted.write.format("com.databricks.spark.csv")\
#    .option("header", "true")\
#    .save(file)

#### 2.2.4 Decision Tree Cross Validation Fitting

In [34]:
# Let's just reuse our CrossValidator with the new dtPipeline, RegressionEvaluator regEval, and 3 fold cross validation
crossval.setEstimator(dtPipeline)

# Let's tune over our dt.maxDepth parameter on the values 2 and 3, create a paramter grid using the ParamGridBuilder
paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [3,4,5])
             .build())

# Add the grid to the CrossValidator
crossval.setEstimatorParamMaps(paramGrid)

# Now let's find and return the best model
dtModel = crossval.fit(trainingSetDF).bestModel

#### 2.2.5 Decision Tree Cross Validation Evaluation

In [36]:
# evaluation
resultsDF = dtModel.transform(testSetDF)
# Create an RMSE evaluator using the label and predicted columns
regEval = RMSLEEvaluator(predictionCol="predicted_meter_reading", labelCol="meter_reading")

# Run the evaluator on the DataFrame
rmsle = regEval.evaluate(resultsDF)

print("Cross Validated Root Mean Squared Logarithmtic Error: %.2f" % rmsle)

### 2.3 Random Forest
#### 2.3.1 Random Forest Pipeline

In [38]:
from pyspark.ml.regression import RandomForestRegressor

# Create a DecisionTreeRegressor
rf = RandomForestRegressor()

rf.setPredictionCol("predicted_meter_reading")\
  .setLabelCol("meter_reading")\
  .setFeaturesCol("features")\
  .setMaxBins(100)

# Create a Pipeline
rfPipeline = Pipeline()

# Set the stages of the Pipeline
rfPipeline.setStages([vectorizer, rf])

#### 2.3.2 Random Forest Simple Fitting

In [40]:
# Let's first train on the entire dataset to see what we get
rfModel = rfPipeline.fit(trainingSetDF)

#### 2.3.3 Random Forest Simple Evaluation

In [42]:
resultsDF = rfModel.transform(testSetDF)
# Create an RMSE evaluator using the label and predicted columns
regEval = RMSLEEvaluator(predictionCol="predicted_meter_reading", labelCol="meter_reading")

# Run the evaluator on the DataFrame
rmsle = regEval.evaluate(resultsDF)

print("Simple Root Mean Squared Logarithmtic Error: %.2f" % rmsle)

#### 2.3.4 Random Forest Cross Validation Fitting

In [44]:
# Let's just reuse our CrossValidator with the new dtPipeline, RegressionEvaluator regEval, and 3 fold cross validation
crossval.setEstimator(rfPipeline)

# Let's tune over our dt.maxDepth parameter on the values 2 and 3, create a paramter grid using the ParamGridBuilder
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [3,4,5])
             .build())

# Add the grid to the CrossValidator
crossval.setEstimatorParamMaps(paramGrid)

# Now let's find and return the best model
rfModel = crossval.fit(trainingSetDF).bestModel

#### 2.3.5 Random Forest Cross Validation Evaluation

In [46]:
# evaluation
resultsDF = rfModel.transform(testSetDF)
# Create an RMSE evaluator using the label and predicted columns
regEval = RMSLEEvaluator(predictionCol="predicted_meter_reading", labelCol="meter_reading")

# Run the evaluator on the DataFrame
rmsle = regEval.evaluate(resultsDF)

print("Cross Validated Root Mean Squared Logarithmtic Error: %.2f" % rmsle)

### 2.4 Gradient-Boosted Trees
#### 2.4.1 Gradient-Boosted Trees Pipeline

In [48]:
from pyspark.ml.regression import GBTRegressor

# Create a DecisionTreeRegressor
gbt = GBTRegressor()

gbt.setPredictionCol("predicted_meter_reading")\
  .setLabelCol("meter_reading")\
  .setFeaturesCol("features")\
  .setMaxBins(100)\
  .setMaxIter(20)

# Create a Pipeline
gbtPipeline = Pipeline()

# Set the stages of the Pipeline
gbtPipeline.setStages([vectorizer, gbt])

#### 2.4.2 Gradient-Boosted Trees Simple Fitting

In [50]:
# Let's first train on the entire dataset to see what we get
gbtModel = gbtPipeline.fit(trainingSetDF)

#### 2.4.3 Gradient-Boosted Trees Simple Evaluation

In [52]:
resultsDF = gbtModel.transform(testSetDF)
# Create an RMSE evaluator using the label and predicted columns
regEval = RMSLEEvaluator(predictionCol="predicted_meter_reading", labelCol="meter_reading")

# Run the evaluator on the DataFrame
rmsle = regEval.evaluate(resultsDF)

print("Simple Root Mean Squared Logarithmtic Error: %.2f" % rmsle)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-657403341536024> in <module> 
 ----> 1 resultsDF = gbtModel . transform ( testSetDF ) 
 2 # Create an RMSE evaluator using the label and predicted columns 
 3 regEval = RMSLEEvaluator ( predictionCol = "predicted_meter_reading" , labelCol = "meter_reading" ) 
 4 
 5 # Run the evaluator on the DataFrame 

 NameError : name 'gbtModel' is not defined

#### 2.4.4 Gradient-Boosted Trees Cross Validation Fitting

In [54]:
# Let's just reuse our CrossValidator with the new dtPipeline, RegressionEvaluator regEval, and 3 fold cross validation
crossval.setEstimator(gbtPipeline)

# Let's tune over our dt.maxDepth parameter on the values 2 and 3, create a paramter grid using the ParamGridBuilder
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [3,4,5])
             .build())

# Add the grid to the CrossValidator
crossval.setEstimatorParamMaps(paramGrid)

# Now let's find and return the best model
gbtModel = crossval.fit(trainingSetDF).bestModel

#### 2.4.5 Gradient-Boosted Trees Cross Validation Evaluation

In [56]:
# evaluation
resultsDF = gbtModel.transform(testSetDF)
# Create an RMSE evaluator using the label and predicted columns
regEval = RMSLEEvaluator(predictionCol="predicted_meter_reading", labelCol="meter_reading")

# Run the evaluator on the DataFrame
rmsle = regEval.evaluate(resultsDF)

print("Cross Validated Root Mean Squared Logarithmtic Error: %.2f" % rmsle)

### 3. Ensemble Model

#### 3.1 Data Preparation

In [59]:
# lrModel
# dtModel
# rfModel
# gbtModel
from pyspark.sql.functions import monotonically_increasing_id 

lrTrainingSetDF = lrModel.transform(trainingSetDF).withColumnRenamed('predicted_meter_reading', 'lr_predicted').select('lr_predicted', "meter_reading")\
                          .withColumn('id', monotonically_increasing_id())
dtTrainingSetDF = dtModel.transform(trainingSetDF).withColumnRenamed('predicted_meter_reading', 'dt_predicted').select('dt_predicted')\
                          .withColumn('id', monotonically_increasing_id())
rfTrainingSetDF = rfModel.transform(trainingSetDF).withColumnRenamed('predicted_meter_reading', 'rf_predicted').select('rf_predicted')\
                          .withColumn('id', monotonically_increasing_id())
gbtTrainingSetDF = gbtModel.transform(trainingSetDF).withColumnRenamed('predicted_meter_reading', 'gbt_predicted').select('gbt_predicted')\
                          .withColumn('id', monotonically_increasing_id())

lrTestSetDF = lrModel.transform(testSetDF).withColumnRenamed('predicted_meter_reading', 'lr_predicted').select('lr_predicted')\
                          .withColumn('id', monotonically_increasing_id())
dtTestSetDF = dtModel.transform(testSetDF).withColumnRenamed('predicted_meter_reading', 'dt_predicted').select('dt_predicted')\
                          .withColumn('id', monotonically_increasing_id())
rfTestSetDF = rfModel.transform(testSetDF).withColumnRenamed('predicted_meter_reading', 'rf_predicted').select('rf_predicted')\
                          .withColumn('id', monotonically_increasing_id())
gbtTestSetDF = gbtModel.transform(testSetDF).withColumnRenamed('predicted_meter_reading', 'gbt_predicted').select('gbt_predicted', "meter_reading")\
                          .withColumn('id', monotonically_increasing_id())

In [60]:
from pyspark.sql.types import StructType

# schema = StructType([])
# leTrainingSetDF = sqlContext.createDataFrame(sc.emptyRDD(), schema)
# leTrainingSetDF = leTrainingSetDF.join(lrTrainingSetDF, how="left_outer")
leTrainingSetDF = lrTrainingSetDF.join(dtTrainingSetDF, "id")
leTrainingSetDF = leTrainingSetDF.join(rfTrainingSetDF, "id")
leTrainingSetDF = leTrainingSetDF.join(gbtTrainingSetDF, "id")
leTrainingSetDF = leTrainingSetDF.drop('id')

# leTestSetDF = sqlContext.createDataFrame(sc.emptyRDD(), schema)
# leTestSetDF = leTestSetDF.join(lrTrainingSetDF, how="left_outer")
leTestSetDF = lrTestSetDF.join(dtTestSetDF, "id")
leTestSetDF = leTestSetDF.join(rfTestSetDF, "id")
leTestSetDF = leTestSetDF.join(gbtTestSetDF, "id")
leTestSetDF = leTestSetDF.drop('id')

leVectorizer = VectorAssembler()
leVectorizer.setInputCols(["lr_predicted", "dt_predicted", "rf_predicted", "gbt_predicted"])
leVectorizer.setOutputCol("features")

#### 3.2 Linear Ensemble
##### 3.2.1 Linear Ensemble PipeLine Set Up

In [62]:
le = LinearRegression(fitIntercept=False)

# Now we set the parameters for the method
le.setPredictionCol("predicted_meter_reading")\
  .setLabelCol("meter_reading")\
  .setMaxIter(100)\
  .setRegParam(0.15)

lePipeline = Pipeline()

lePipeline.setStages([leVectorizer, le])

##### 3.2.2 Linear Ensemble Simple Fitting

In [64]:
leModel = lePipeline.fit(leTrainingSetDF)

##### 3.2.3 Linear Ensemble Simple Evaluation

In [66]:
# The intercept is as follows:
# intercept = leModel.stages[1].intercept

# The coefficents (i.e., weights) are as follows:

weights = leModel.stages[1].coefficients

# Create a list of the column names (without PE)
featuresNoLabel = [col for col in leTrainingSetDF.columns if col != "meter_reading"]

# Merge the weights and labels
coefficents = zip(weights, featuresNoLabel)

#equation = "y = {intercept}".format(intercept=intercept)
variables = []
for x in coefficents:
    weight = abs(x[0])
    name = x[1]
    symbol = "+" if (x[0] > 0) else "-"
    equation += (" {} ({} * {})".format(symbol, weight, name))

print("Linear Regression Equation: " + equation)

resultsDF = leModel.transform(leTestSetDF)
# Create an RMSE evaluator using the label and predicted columns
regEval = RMSLEEvaluator(predictionCol="predicted_meter_reading", labelCol="meter_reading")

# Run the evaluator on the DataFrame
rmsle = regEval.evaluate(resultsDF)

print("Simple Root Mean Squared Logarithmtic Error: %.2f" % rmsle)
